# D. State stabilization in a noisy environment with Jaynes-Cummings controls

Here the result is as follows: The algorithm optimizes the params, such that the POVM always outputs 1, implying that the measurement leaves the target state invariant. this is what we are indeed seeing when printing the measurement outcome and its probability, when batching however, the optimizer struggles to converge.

Also, lookup here is much better than nn for the same hyperparameters.

This is actually a perfect example, of feedback grape modifying the params so that a certain measurement sequence will always be output because this measurement sequence is the one that is going to lead to the best fidelity

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
from feedback_grape.fgrape import optimize_pulse_with_feedback
from feedback_grape.utils.operators import (
    sigmap,
    sigmam,
    create,
    destroy,
    identity,
    cosm,
    sinm,
)
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp
from jax.scipy.linalg import expm

In [3]:
N_cav = 30

## Here, dividing alpha into real and imaginary parts complicates the optimization and converges at 0.89 while if we do not use the imaginary part it converges at 0.999

In [4]:
def qubit_unitary(alpha_re):
    alpha = alpha_re
    return tensor(
        identity(N_cav),
        expm(-1j * (alpha * sigmap() + alpha.conjugate() * sigmam()) / 2),
    )

In [5]:
def qubit_cavity_unitary(beta_re):
    beta = beta_re
    return expm(
        -1j
        * (
            beta * (tensor(destroy(N_cav), sigmap()))
            + beta.conjugate() * (tensor(create(N_cav), sigmam()))
        )
        / 2
    )

In [6]:
from feedback_grape.utils.operators import create, destroy


def povm_measure_operator(measurement_outcome, params):
    """
    POVM for the measurement of the cavity state.
    returns Mm ( NOT the POVM element Em = Mm_dag @ Mm ), given measurement_outcome m, gamma and delta
    """
    gamma, delta = params
    number_operator = tensor(create(N_cav) @ destroy(N_cav), identity(2))
    angle = (gamma * number_operator) + delta / 2
    meas_op = jnp.where(
        measurement_outcome == 1,
        cosm(angle),
        sinm(angle),
    )
    return meas_op

In [7]:
from feedback_grape.utils.states import coherent

alpha = 3
psi_target = tensor(
    coherent(N_cav, alpha)
    + coherent(N_cav, -alpha)
    + coherent(N_cav, 1j * alpha)
    + coherent(N_cav, -1j * alpha),
    basis(2),
)  # 4-legged state

# Normalize psi_target before constructing rho_target
psi_target = psi_target / jnp.linalg.norm(psi_target)
rho_target = psi_target @ psi_target.conj().T

In [8]:
rho_target.shape

(60, 60)

### It is important to test what the POVM probability is, to check if your state is normalized. if the probability is bounded between 0 and 1 then normalized

In [9]:
# TODO/Question: should one normalize within the optimization just in case?
from feedback_grape.utils.povm import (
    _probability_of_a_measurement_outcome_given_a_certain_state,
)

_probability_of_a_measurement_outcome_given_a_certain_state(
    rho_target, 1, povm_measure_operator, [0.058, jnp.pi / 2]
)

Array(0.70390199, dtype=float64)

In [10]:
from feedback_grape.utils.fidelity import fidelity

print(fidelity(U_final=rho_target, C_target=rho_target, evo_type="density"))

1.000000011680078


### Without dissipation

In [11]:
# Here the loss directly corressponds to the -fidelity (when converging) because log(1) is 0 and
# the algorithm is choosing params that makes the POVM generate prob = 1
from feedback_grape.fgrape import Gate, Decay

measure = Gate(
    gate=povm_measure_operator,
    initial_params=[0.058, jnp.pi / 2],  # gamma and
    measurement_flag=True,
)

qub_unitary = Gate(
    gate=qubit_unitary,
    initial_params=[jnp.pi / 3],  # alpha
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)

qub_cav = Gate(
    gate=qubit_cavity_unitary,
    initial_params=[jnp.pi / 3],  # beta
    measurement_flag=False,
    # "param_constraints": [[0, 0.5], [-1, 1]],
)


system_params = [measure, qub_unitary, qub_cav]
result = optimize_pulse_with_feedback(
    U_0=rho_target,
    C_target=rho_target,
    system_params=system_params,
    num_time_steps=1,
    mode="lookup",
    goal="fidelity",
    max_iter=1000,
    convergence_threshold=1e-16,
    learning_rate=0.02,
    evo_type="density",
    batch_size=1,
)

In [12]:
# 0.999995162213917
print(result.final_fidelity)

0.999995162213917


In [13]:
result.optimized_trainable_parameters["initial_params"]

[Array([1.06202532e-09, 1.46607657e+00], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]

In [14]:
result.returned_params

[[Array([[1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00],
         [1.06202532e-09, 1.46607657e+00]], dtype=float64),
  Array([[-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567],
         [-0.00622567]], dtype=float64),
  Array([[-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05],
         [-4.05896188e-05]], dtype=float64)]

In [15]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.000000011680078
fidelity of state 0: 0.9999951622139169
fidelity of state 1: 0.9999951622139169
fidelity of state 2: 0.9999951622139169
fidelity of state 3: 0.9999951622139169
fidelity of state 4: 0.9999951622139169
fidelity of state 5: 0.9999951622139169
fidelity of state 6: 0.9999951622139169
fidelity of state 7: 0.9999951622139169
fidelity of state 8: 0.9999951622139169
fidelity of state 9: 0.9999951622139169


### With Dissipation

In [16]:
# Note if tsave = jnp.linspace(0, 1, 1) = [0.0] then the decay is not applied ?
# because the first time step has the original non decayed state
# Question: Here I should not use any sort of Hamiltonian or tspan or something?
decay = Decay(
    c_ops=[tensor(identity(N_cav), jnp.sqrt(0.15) * sigmam())],
)
# not supported to have decay at very end, not supported to have multiple consecutive decays
# TODO: use system_params in calculate time step instead: 
system_params = [decay, measure, qub_unitary, qub_cav]
result = optimize_pulse_with_feedback(
    U_0=rho_target,
    C_target=rho_target,
    system_params=system_params,
    num_time_steps=1,
    mode="lookup",
    goal="fidelity",
    max_iter=1000,
    convergence_threshold=1e-6,
    learning_rate=0.02,
    evo_type="density",
    batch_size=1,
)

In [17]:
print(result.final_fidelity)

0.9277299964479836


In [18]:
from feedback_grape.utils.fidelity import fidelity

print(
    "initial fidelity:",
    fidelity(C_target=rho_target, U_final=rho_target, evo_type="density"),
)
for i, state in enumerate(result.final_state):
    print(
        f"fidelity of state {i}:",
        fidelity(C_target=rho_target, U_final=state, evo_type="density"),
    )

initial fidelity: 1.000000011680078
fidelity of state 0: 0.9277299964479836
fidelity of state 1: 0.9277299964479836
fidelity of state 2: 0.9277299964479836
fidelity of state 3: 0.9277299964479836
fidelity of state 4: 0.9277299964479836
fidelity of state 5: 0.9277299964479836
fidelity of state 6: 0.9277299964479836
fidelity of state 7: 0.9277299964479836
fidelity of state 8: 0.9277299964479836
fidelity of state 9: 0.9277299964479836


In [19]:
result.returned_params

[[Array([[5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00],
         [5.05638126e-04, 1.67543401e+00]], dtype=float64),
  Array([[-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678],
         [-0.00195678]], dtype=float64),
  Array([[0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197],
         [0.00336197]], dtype=float64)]]

In [20]:
result.optimized_trainable_parameters["initial_params"]

[Array([5.05638126e-04, 1.67543401e+00], dtype=float64),
 Array([1.04719755], dtype=float64),
 Array([1.04719755], dtype=float64)]